# Project 02 - Image Processing

## Student Information

- Full name: Tô Quốc Thanh
- Student ID: 22127388
- Class: 22CLC10

## Required Libraries

In [1]:
# IMPORT YOUR LIBS HERE
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

## Function Definitions

In [2]:
def read_img(img_path):
    '''
    Read image from img_path

    Parameters
    ----------
    img_path : str
        Path of image

    Returns
    -------
        Image
    '''

    # YOUR CODE HERE
    img = Image.open(img_path)
    return np.array(img).astype(np.uint8)


def show_img(img):
    '''
    Show image

    Parameters
    ----------
    img : 2D array
        Image
    '''

    # YOUR CODE HERE
    plt.imshow(img, cmap = 'gray')

def save_img(img, img_path):
    '''
    Save image to img_path

    Parameters
    ----------
    img : 2D array
        Image
    img_path : str
        Path of image
    '''

    # YOUR CODE HERE
    if img.dtype != np.uint8:
        img = img.astype(np.uint8)

    pil_img = Image.fromarray(img)
    
    pil_img.save(img_path)

#--------------------------------------------------------------------------------
# YOUR FUNCTIONS HERE

def change_light(img, factor):
    '''
    Change the light of the image

    Parameters
    ----------
    img : 2D
        Image
    factor : float
        Factor to change light of the image
    Returns
    -------
        Image
    '''

    # YOUR CODE HERE
    temp = img.astype(float) #convert to float to avoid overflow, because uint8 has range from 0 to 255
    new_img = np.clip(temp + factor, 0, 255).astype(np.uint8)
    return new_img

def change_contrast(img, factor):
    '''
    Change the contrast of the image

    Parameters
    ----------
    img : 2D
        Image
    factor : float
        Factor to change contrast of the image

    Returns
    -------
        Image
    '''

    # YOUR CODE HERE
    temp = img.astype(float) #convert to float to avoid overflow, because uint8 has range from 0 to 255
    new_img = np.clip(temp * factor, 0, 255).astype(np.uint8)
    return new_img

def flip_image(img, type):
    '''
    Flip the image

    Parameters
    ----------
    img : 2D
        Image
    type : int
        1: horizontally, 2: vertically
    Returns
    -------
        Image
    '''

    # YOUR CODE HERE
    if type == 1:
        return img[:, ::-1]
    elif type == 2:
        return img[::-1, :]

def convert_to_gray(img):
    '''
    Convert the image to gray

    Parameters
    ----------
    img : 2D
        Image

    Returns
    -------
        Image
    '''

    # YOUR CODE HERE
    return np.dot(img, [0.2989, 0.5870, 0.1140])


def convert_to_sepia(img):
    '''
    Convert the image to sepia

    Parameters
    ----------
    img : 2D
        Image

    Returns
    -------
        Image
    '''

    # YOUR CODE HERE
    
    sepia_filter = np.array([[0.393, 0.769, 0.189],
                             [0.349, 0.686, 0.168],
                             [0.272, 0.534, 0.131]])
    sepia_img = np.dot(img, sepia_filter.T)
    
    sepia_img = np.clip(sepia_img, 0, 255).astype(np.uint8)
    return sepia_img

def create_blur_image(img):
    '''
    Create the blur image

    Parameters
    ----------
    img : 2D
        Image

    Returns
    -------
        Image
    '''

    # YOUR CODE HERE

    gray_img = convert_to_gray(img)
    height, width = gray_img.shape
    
    kernel = np.ones((3, 3))/9
    
    result = np.zeros((height, width))
    
    for i in range(1, height - 1):
        for j in range(1, width - 1):
            result[i, j] = np.sum(gray_img[i-1:i+1+1, j-1:j+1+1] * kernel)
    
    return result.astype(np.uint8)
    
def create_sharpen_image(img):
    '''
    Create the sharpen image

    Parameters
    ----------
    img : 2D
        Image

    Returns
    -------
        Image
    '''

    # YOUR CODE HERE
    
    gray_img = convert_to_gray(img)
    height, width = gray_img.shape
    kernel = np.array([[ 0, -1,  0],
                       [-1,  5, -1],
                       [ 0, -1,  0]])
    
    result = np.zeros((height, width)).astype(float)
    
    for i in range(1, height - 1):
        for j in range(1, width - 1):
            result[i, j] = np.sum(gray_img[i-1:i+1+1, j-1:j+1+1] * kernel)
    
    return result.clip(0, 255).astype(np.uint8)

def crop_image(img):
    '''
    Crop the image

    Parameters
    ----------
    img : 2D
        Image

    Returns
    -------
        Image
    '''

    # YOUR CODE HERE
    
    height, width, _ = img.shape
    
    return img[height//4:3*height//4, width//4:3*width//4]
    
def circle_crop_image(img):
    '''
    Circle crop the image

    Parameters
    ----------
    img : 2D
        Image

    Returns
    -------
        Image
    '''

    # YOUR CODE HERE
    height, width, _ = img.shape
    center = (width / 2, height / 2)
    radius = min(center[0], center[1])
    circle_img = img.copy()
    for i in range(height):
        for j in range(width):
            if (i - center[1])**2 + (j - center[0])**2 > radius**2:
                circle_img[i, j] = 0
    return circle_img
        

def two_elips_crop_image(img):
    '''
    Two elips crop the image

    Parameters
    ----------
    img : 2D
        Image

    Returns
    -------
        Image
    '''

    # YOUR CODE HERE
    elips_img = img.copy()
    
    height, width, _ = img.shape
    length = height
    center = (length / 2, length / 2)
    
    F1 = (length / 9, length / 9)
    F2 = (length / 9 * 8, length / 9 * 8)
    #F1F2 = 2c -> F1F2 bigger, b will be small (b^2 = a^2 - c^2), elip will be thinner

    
    offset = length / 5 * 4
    M = (offset, length)

    MF1 = np.sqrt((M[0] - F1[0])**2 + (M[1] - F1[1])**2)
    MF2 = np.sqrt((M[0] - F2[0])**2 + (M[1] - F2[1])**2)
    F1F2 = np.sqrt((F1[0] - F2[0])**2 + (F1[1] - F2[1])**2)
    
    a = (MF1 + MF2) / 2
    c = F1F2 / 2
    b = np.sqrt(a**2 - c**2)
    
    angle1 = np.deg2rad(45)
    angle2 = np.deg2rad(-45)
    
    cos_angle1 = np.cos(angle1)
    sin_angle1 = np.sin(angle1)
    
    cos_angle2 = np.cos(angle2)
    sin_angle2 = np.sin(angle2)
    
    for i in range(height):
        for j in range(width):
            
            x = j - center[0]
            y = i - center[1]
            
            #xem center là gốc tọa độ của elip này
            
            x1 = x * cos_angle1 - y * sin_angle1
            y1 = x * sin_angle1 + y * cos_angle1
            x2 = x * cos_angle2 - y * sin_angle2
            y2 = x * sin_angle2 + y * cos_angle2
            
            if (x1**2 / a**2 + y1**2 / b**2 > 1) and (x2**2 / a**2 + y2**2 / b**2 > 1):
                elips_img[i, j] = 0

    return elips_img

def zoom_image(img, factor):
    '''
    Zoom in the image

    Parameters
    ----------
    img : 2D
        Image
    factor : float
        Factor to zoom in/ out
    Returns
    -------
        Image
    '''

    # YOUR CODE HERE
    height, width, _ = img.shape
    zoom_image = np.zeros((int(height*factor), int(width*factor), 3))
    for i in range(int(height*factor)):
        for j in range(int(width*factor)):
            zoom_image[i, j] = img[int(i/factor), int(j/factor)]
    zoom_image = np.clip(zoom_image, 0, 255).astype(np.uint8)
    return zoom_image

<ins>Note:</ins> For clarity, include docstrings with each function.

## Your tests

In [ ]:
# YOUR CODE HERE
input_img_path = 'img2.jpg'
# output_img_path = 'new_img.jpg'

# # Open an img file
img_2D = read_img(input_img_path)
# print(img_2D)

# ----------------- Change light -----------------
# increase_light = change_light(img_2D, 50)
# decrease_light = change_light(img_2D, -50)
# show_img(increase_light)
# show_img(decrease_light)

# ----------------- Change contrast -----------------
# increase_contrast = change_contrast(img_2D, 2)
# decrease_contrast = change_contrast(img_2D, 0.5)
# show_img(increase_contrast)
# show_img(decrease_contrast)

#-------------------Flip image--------------------
# flip_horizontally = flip_image(img_2D, 1)
# flip_vertically = flip_image(img_2D, 2)
# show_img(flip_horizontally)
# show_img(flip_vertically)

#-------------------Convert to gray--------------------
# gray_img = convert_to_gray(img_2D)
# show_img(gray_img)
# save_img(gray_img, output_img_path)

# -------------------Convert to sepia--------------------
# sepia_img = convert_to_sepia(img_2D)
# show_img(sepia_img)

# -------------------Create blur image--------------------
# blur_img = create_blur_image(img_2D)
# show_img(blur_img)

# -------------------Create sharpen image--------------------
# sharpen_img = create_sharpen_image(img_2D)
# show_img(sharpen_img)
# save_img(sharpen_img, output_img_path)

# --Crop image--
# crop_img = crop_image(img_2D)
# show_img(crop_img)

# --Circle crop image--
# circle_crop_img = circle_crop_image(img_2D)
# show_img(circle_crop_img)

# --Two elips crop image--
# two_elips_crop_img = two_elips_crop_image(img_2D)
# show_img(two_elips_crop_img)

# --Zoom in image--
# zoom_in_img = zoom_image(img_2D, 2)
# show_img(zoom_in_img)
# save_img(zoom_in_img, output_img_path)
# show_img(img_2D)

## Main FUNCTION

In [119]:
# YOUR CODE HERE
def main():
    input_img_path = input('Enter the path of the image: ')
    
    try:
        img_2D = read_img(input_img_path)
    except:
        print('Cannot read the image')
        return
    
    print("--------------------")
    print('0. All functions')
    print('1. Change light')
    print('2. Change contrast')
    print('3. Flip image')
    print('4. Convert to gray/ sepia')
    print('5. Create blur/ sharpen image')
    print('6. Crop image')
    print('7. Shape crop image')
    print('8. Zoom image')
    print("--------------------")
    num = int(input('Enter the number of the function you want to use: '))
    

    if num == 1 or num == 0:
        num1 = 0
        factor = 50
        if num == 1:
            print('1. Lighter')
            print('2. Darker')
            while num1 != 1 and num1 != 2:
                num1 = int(input('Please enter the number between 1 and 2: '))
            factor = float(input('Enter the factor you want to change light: '))
        if num1 == 1 or num == 0:
            img_lighter = change_light(img_2D, np.abs(factor))
            save_img(img_lighter, '1.1_lighter.jpg')
            if num1 == 1:
                show_img(img_lighter)
        if num1 == 2 or num == 0:
            img_darker = change_light(img_2D, -np.abs(factor))
            save_img(img_darker, '1.2_darker.jpg')
            if num1 == 2:
                show_img(img_darker)
            
    if num == 2 or num == 0:
        num2 = 0
        factor = 2
        if num == 2:
            print('1. Increase contrast')
            print('2. Decrease contrast')
            num2 = int(input('Please enter the number between 1 and 2: '))
            factor = float(input('Enter the factor you want to change contrast: '))
        if num2 == 1 or num == 0:
            increase_contrast = change_contrast(img_2D, factor)
            save_img(increase_contrast, '2.1_increase_contrast.jpg')
            if num2 == 1:
                show_img(increase_contrast)
        if num2 == 2 or num == 0:
            decrease_contrast = change_contrast(img_2D, 1/factor)
            save_img(decrease_contrast, '2.2_decrease_contrast.jpg')
            if num2 == 2:
                show_img(decrease_contrast)
        
    if num == 3 or num == 0:
        num3 = 0
        if num == 3:
            print('1. Horizontally flip')
            print('2. Vertically flip')
            num3 = int(input('Please enter the number between 1 and 2: '))
        if num3 == 1 or num == 0:
            horizontally_flipped_image = flip_image(img_2D, 1)
            save_img(horizontally_flipped_image, '3.1_horizontally_flipped_image.jpg')
            if num3 == 1:
                show_img(vertically_flipped_image)
        if num3 == 2 or num == 0:
            vertically_flipped_image= flip_image(img_2D, 2)
            save_img(vertically_flipped_image, '3.2_vertically_flipped_image.jpg')
            if num3 == 2:
                show_img(horizontally_flipped_image)
            
        
    if num == 4 or num == 0:
        num4 = 0
        if num == 4:
            print('1. Convert to gray')
            print('2. Convert to sepia')
            num4 = int(input('Please enter the number between 1 and 2: '))
        if num4 == 1  or num == 0:
            gray_img = convert_to_gray(img_2D)
            save_img(gray_img, '4.1_gray_img.jpg')
            if num4 == 1:
                show_img(gray_img)
        if num4 == 2 or num == 0:
            sepia_img = convert_to_sepia(img_2D)
            save_img(sepia_img, '4.2_sepia_img.jpg')
            if num4 == 2:
                show_img(sepia_img)
            
    if num == 5 or num == 0:
        num5 = 0
        if num == 5:
            print('1. Create blur image')
            print('2. Create sharpen image')
            num5 = int(input('Please enter the number between 1 and 2: '))
        if num5 == 1 or num == 0:
            blur_img = create_blur_image(img_2D)
            save_img(blur_img, '5.1_blur_img.jpg')
            if num5 == 1:
                show_img(blur_img)
        if num5 == 2 or num == 0:
            sharpen_img = create_sharpen_image(img_2D)
            save_img(sharpen_img, '5.2_sharpen_img.jpg')
            if num5 == 2:
                show_img(sharpen_img)
        
    if num == 6 or num == 0:
        crop_img = crop_image(img_2D)
        save_img(crop_img, '6_crop_img.jpg')
        if num == 6:
            show_img(crop_img)
        
    if num == 7 or num == 0:
        num7 = 0
        if num == 7:
            print('1. Circle crop image')
            print('2. Two elips crop image')
            num7 = int(input('Please enter the number between 1 and 2: '))
        if num7 == 1 or num == 0:
            circle_crop_img = circle_crop_image(img_2D)
            save_img(circle_crop_img, '7.1_circle_crop_img.jpg')
            if num7 == 1:
                show_img(circle_crop_img)
        if num7 == 2 or num == 0:
            two_elips_crop_img = two_elips_crop_image(img_2D)
            save_img(two_elips_crop_img, '7.2_two_elips_crop_img.jpg')
            if num7 == 2:
                show_img(two_elips_crop_img)
    
    if num == 8 or num == 0:
        num8 = 0
        factor = 2
        if num == 8:
            print('1. Zoom in image')
            print('2. Zoom out image')
            num8 = int(input('Please enter the number between 1 and 2: '))
            factor = float(input('Enter the factor you want to zoom in/ out: '))
        if num8 == 1 or num == 0:
            zoom_in = zoom_image(img_2D, factor)
            save_img(zoom_in, '8.1_zoom_in.jpg')
            if num8 == 1:
                show_img(zoom_in)
        if num8 == 2 or num == 0:
            zoom_out = zoom_image(img_2D, 1/factor)
            save_img(zoom_out, '8.2_zoom_out.jpg')
            if num8 == 2:
                show_img(zoom_out)
    print('Done!')
    
    


In [ ]:
# Call main function
if __name__ == "__main__":
    main()